In [92]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [93]:
k_data = json.load(open("./KOK-VQA.json"))
k_df = pd.DataFrame(k_data["annotations"])

In [94]:
k_df["questions"]

0    [{'question_id': '1-1', 'question_ko': '사진속 어떤...
1    [{'question_id': '2-1', 'question_ko': '사진속 어떤...
2    [{'question_id': '1-1', 'question_ko': '이미지의 어...
Name: questions, dtype: object

In [95]:
k_kb = json.load(open("./KOK-VQA-KB.json"))
kb_df = pd.DataFrame(k_kb["knowledgebase"])

In [96]:
kb_df.T

,KB,context,dataset,e1,e1_label,e2,e2_label,r,sources,surface_en,surface_ko
ko.dbpedia/1137,ko.dbpedia,None,None,http://ko.dbpedia.org/resource/피아노,피아노,피아노,피아노,http://dbpedia.org/ontology/title,None,None,"[피아노, 이름, 피아노]"
ko.dbpedia/1142,ko.dbpedia,None,None,http://ko.dbpedia.org/resource/피아노,피아노,None,바르톨로메오,http://dbpedia.org/ontology/abstract,None,None,"[피아노, 발명가, 크리스토포리]"
ko.dbpedia/1437,ko.dbpedia,None,None,http://ko.dbpedia.org/resource/탁구,탁구,http://ko.dbpedia.org/resource/탁구라켓,탁구라켓,http://dbpedia.org/ontology/equipment,None,None,"[탁구, 장비, 탁구라켓]"
conceptnet/1,conceptnet,None,/d/conceptnet/4/en,/c/en/piano/r,piano,/c/en/very_popular_musical_instrument,very popular musical instrument,/r/IsA,None,None,"[피아노, 이다, 매우 인기있는 악기]"
conceptnet/2,conceptnet,None,/d/conceptnet/4/en,/c/en/piano/r,piano,/c/en/music_store,music store,/r/AtLocation,None,None,"[피아노, 위치하다, 음반 가게]"


## 3. one-cycle EDA check

In [147]:
k_one = json.load(open("./one_cycle_samples.json"))
k_df_one = pd.DataFrame(k_one["annotations"])

In [148]:
k_df_one.head(2)

,ACTION,CAPTION,HEIGHT,IMAGE_ACQUISITION_DATE,IMAGE_ID,IMAGE_NAME,IMAGE_URL,LICENSE,MAINOBJECT,MAINOBJECT_URL,SCENE,SUBOBJECT_1,SUBOBJECT_2,WIDTH,WIKI_FILE,bounding_box,questions
0,Lie,바닥에 돌이 있다.,600,20200929,120100120220622135110,120100120220622135110.jpg,https://upload.wikimedia.org/wikipedia/commons...,FREE,개미,http://ko.dbpedia.org/resource/개미,forest - broadleaf,자갈,헛소리,800,1932Ants of the Philippines eating Alimango 33...,"[{'BOX_ID': '629421', 'OBJECT': '개미', 'X_COORD...","[{'question_id': '120100120220622135110-1', 'q..."
1,Lie,창문 밖에 나무들이 많이 있다.,768,20090531,110500120220627131757,110500120220627131757.jpg,https://upload.wikimedia.org/wikipedia/commons...,FREE,창_(건축),http://ko.dbpedia.org/resource/창_(건축),house,유리 바구니,오리,576,Window's.jpg,"[{'BOX_ID': '629403', 'OBJECT': '창_(건축)', 'X_C...","[{'question_id': '110500120220627131757-1', 'q..."


In [149]:
k_df_one.columns

Index(['ACTION ', 'CAPTION ', 'HEIGHT ', 'IMAGE_ACQUISITION_DATE', 'IMAGE_ID ',
       'IMAGE_NAME ', 'IMAGE_URL ', 'LICENSE ', 'MAINOBJECT ',
       'MAINOBJECT_URL ', 'SCENE ', 'SUBOBJECT_1 ', 'SUBOBJECT_2 ', 'WIDTH ',
       'WIKI_FILE ', 'bounding_box', 'questions'],
      dtype='object')

### 3-1. preprocessing

In [150]:
k_df_one["question1"] = k_df_one["questions"].apply(lambda x: x[0]["question_ko"])
k_df_one["question2"] = k_df_one["questions"].apply(lambda x: x[1]["question_ko"])

k_df_one["answer1"] = k_df_one["questions"].apply(lambda x: x[0]["answer_ko"])
k_df_one["answer2"] = k_df_one["questions"].apply(lambda x: x[1]["answer_ko"])

k_df_one["fact1"] = k_df_one["questions"].apply(lambda x: x[0]["fact"][0])
k_df_one["fact2"] = k_df_one["questions"].apply(lambda x: x[1]["fact"][0])

k_df_one["bbox"] = k_df_one["bounding_box"].apply(lambda x: x[0]["OBJECT"])

del(k_df_one["questions"])
del(k_df_one["bounding_box"])


In [155]:
k_df_one.head(10)

,ACTION,CAPTION,HEIGHT,IMAGE_ACQUISITION_DATE,IMAGE_ID,IMAGE_NAME,IMAGE_URL,LICENSE,MAINOBJECT,MAINOBJECT_URL,...,SUBOBJECT_2,WIDTH,WIKI_FILE,question1,question2,answer1,answer2,fact1,fact2,bbox
0,Lie,바닥에 돌이 있다.,600,20200929,120100120220622135110,120100120220622135110.jpg,https://upload.wikimedia.org/wikipedia/commons...,FREE,개미,http://ko.dbpedia.org/resource/개미,...,헛소리,800,1932Ants of the Philippines eating Alimango 33...,이미지 속 공생하는 곤충은 무엇일까,이미지 속 게 딱지에 올라가 있는 개미는 몇 마리니,개미,8,ko.dbpedia/197,ko.dbpedia/197,개미
1,Lie,창문 밖에 나무들이 많이 있다.,768,20090531,110500120220627131757,110500120220627131757.jpg,https://upload.wikimedia.org/wikipedia/commons...,FREE,창_(건축),http://ko.dbpedia.org/resource/창_(건축),...,오리,576,Window's.jpg,이미지 속 네모난 건축물은 뭐야,창은 몇 개야,창,1,ko.dbpedia/189,ko.dbpedia/189,창_(건축)
2,Lie,키보드가 놓여있다.,683,20130302,110101820220621113002,110101820220621113002.jpg,https://upload.wikimedia.org/wikipedia/commons...,FREE,키보드_(악기),http://ko.dbpedia.org/resource/키보드_(악기),...,탁자,1024,Steinberg CC121 Cubase control surface and M-A...,다리나 기둥으로 받쳐 만든 가구는 무엇일까,흰색과 검정색 건반으로 이루어져 있는 악기는 무엇일까,탁자,키보드,ko.dbpedia/827,ko.dbpedia/827,키보드_(악기)
3,Lie,장애인 화장실 문 앞에 창살이 있다.,673,20120117,110500320220622150258,110500320220622150258.jpg,https://upload.wikimedia.org/wikipedia/commons...,FREE,화장실,http://ko.dbpedia.org/resource/화장실,...,의자,1280,Public toilet - Haikou 01.jpg,전 세계 인구 중 화장실이 없는 사람은 몇 퍼센트야,이미지 속 화장실 문 중에 닫혀 있는 문은 몇 개야,40,2,ko.dbpedia/191,ko.dbpedia/191,화장실
4,Lie,케이크에 양초가 있다.,600,20050528,110200920220621145512,110200920220621145512.jpg,https://upload.wikimedia.org/wikipedia/commons...,FREE,케이크,http://ko.dbpedia.org/resource/케이크,...,넵킨,792,Birthday Cake2.jpg,이미지 속 후식으로 먹는 프랑스 요리는 무엇이니,이미지 속 케이크에 꽂혀있는 것은 뭐니,케이크,초,ko.dbpedia/149,ko.dbpedia/149,케이크
5,Sit,벌이 꽃잎 위에 앉아있고 옆에 벌레가 있다.,600,20170706,120100220220622152103,120100220220622152103.jpg,https://upload.wikimedia.org/wikipedia/commons...,FREE,벌_(곤충),http://ko.dbpedia.org/resource/벌_(곤충),...,벌레,800,PKLHmyz20824 (20).jpg,벌의 목은 뭐야,이미지 속 벌은 몇 마리니,벌목,1,ko.dbpedia/228,ko.dbpedia/228,벌_(곤충)
6,Play a musical instrument,사람이 바이올린을 연주하고 있다.,795,20110110,110101720220620180934,110101720220620180934.jpg,https://upload.wikimedia.org/wikipedia/commons...,FREE,바이올린,http://ko.dbpedia.org/resource/바이올린,...,마이크로폰,640,Student-loan-violin (5342880285).jpg,바이올린과 대등할 만큼 인기가 많은 악기는 무엇이니,이미지 속 연주자의 악기는 무엇일까,피아노,바이올린,ko.dbpedia/15,ko.dbpedia/15,바이올린
7,Play a musical instrument,무대 위에서 남자가 아코디언을 연주하며 노래를 부르고 있다.,544,20150731,110100120220620135251,110100120220620135251.jpg,https://upload.wikimedia.org/wikipedia/commons...,FREE,아코디언,http://ko.dbpedia.org/resource/아코디언,...,마이크로폰,1024,Garish Heimatsound Festival 2015 (32).jpg,아코디언의 음역은 무엇에 따라 다르니,아코디언은 몇 명이 들고 있어,배치,1,ko.dbpedia/66,ko.dbpedia/66,아코디언
8,Lie,악기들이 놓여져 있다.,1023,20140803,110101220220621102819,110101220220621102819.jpg,https://upload.wikimedia.org/wikipedia/commons...,FREE,오보에,http://ko.dbpedia.org/resource/오보에,...,드럼_세트,682,"Drum and ken saranai (oboe), Cham - Vietnam Mu...",이미지 속 타악기는 뭘까,이미지 속 오보에는 몇 개니,드럼 세트,1,ko.dbpedia/10,ko.dbpedia/10,오보에
9,Lie,신호등이 빨간불이다.,623,20200226,110400220220621150208,110400220220621150208.jpg,https://upload.wikimedia.org/wikipedia/commons...,FREE,소화전,http://ko.dbpedia.org/resource/소화전,...,신호등,1024,WY capitol night2.jpg,소방 설비는 무엇을 진압하는 데 사용할까,이미지 속 소화전은 몇 가지 색으로 이루어져 있니,화재,2,ko.dbpedia/178,ko.dbpedia/178,소화전


In [152]:
eda_list = []
for colum in k_df_one.columns:
    if colum not in ["question1","question2","answer1","answer2", "fact1", "fact2"]:
        num_unique = len(k_df_one[colum].unique())
        unique_ratio = num_unique/len(k_df_one)
        eda_list.append([colum, num_unique, unique_ratio])
        #print(colum, len(k_df_one[colum].unique()), )

num_unique = len(pd.concat([ k_df_one["question1"], k_df_one["question2"] ]).unique())
unique_ratio = (num_unique/(len(k_df_one)*2)) * 100
eda_list.append(["questions", num_unique, unique_ratio])

num_unique = len(pd.concat([ k_df_one["answer1"], k_df_one["answer2"] ]).unique())
unique_ratio = (num_unique/(len(k_df_one)*2)) * 100
eda_list.append(["answers", num_unique, unique_ratio])

num_unique = len(pd.concat([ k_df_one["fact1"], k_df_one["fact2"] ]).unique())
unique_ratio = (num_unique/(len(k_df_one)*2)) * 100
eda_list.append(["facts", num_unique, unique_ratio])


In [153]:
df_eda = pd.DataFrame(eda_list)
df_eda.columns = ["colum_name", "num_unique", "unique_ratio"]

In [154]:
df_eda

,colum_name,num_unique,unique_ratio
0,ACTION,25,0.040984
1,CAPTION,546,0.895082
2,HEIGHT,104,0.170492
3,IMAGE_ACQUISITION_DATE,475,0.778689
4,IMAGE_ID,610,1.000000
5,IMAGE_NAME,610,1.000000
6,IMAGE_URL,610,1.000000
7,LICENSE,1,0.001639
8,MAINOBJECT,32,0.052459
9,MAINOBJECT_URL,32,0.052459


In [157]:
len(k_df_one[k_df_one["fact1"] == k_df_one["fact2"]])

610